Importing modules

In [1]:
#visualisation and processing modules
import numpy as np
import pandas as pd
import requests
import datetime

#threading modules
import logging
import threading
import time
from threading import Thread
from time import sleep, perf_counter

Reading in of Datasets

In [2]:
#Prepare column names
columns1=["Passenger_ID", "Flight_ID", "From_Airport_Code", "To_Airport_Code","Departure_Time", "Duration"]
columns2=["Airport","From_Airport_Code", "Latitude","Longitude"]
#Read in both data sets
airport=pd.read_csv('AComp_Passenger_data_no_error(1).csv',names=columns1)
latlong=pd.read_csv('Top30_airports_LatLong(1).csv',names=columns2)



In [3]:
#Basic overview of airport dataset
print(airport) 

    Passenger_ID Flight_ID From_Airport_Code To_Airport_Code  Departure_Time  \
0     UES9151GS5  SQU6245R               DEN             FRA      1420564460   
1     UES9151GS5  XXQ4064B               JFK             FRA      1420563917   
2     EZC9678QI6  SOH3431A               ORD             MIA      1420563649   
3     ONL0812DH1  SOH3431A               ORD             MIA      1420563649   
4     CYJ0225CH1  PME8178S               DEN             PEK      1420564409   
..           ...       ...               ...             ...             ...   
495   BWI0520BG6  BER7172M               KUL             LAS      1420565167   
496   LLZ3798PE3  EWH6301Y               CAN             DFW      1420564967   
497   KKP5277HZ7  KJR6646J               IAH             BKK      1420565203   
498   JJM4724RF7  XXQ4064B               JFK             FRA      1420563917   
499   SJD8775RZ4  WSK1289Z               CLT             DEN      1420563542   

     Duration  
0        1049  
1      

In [4]:
#Basic overview of latitude and logitude dataset
print(latlong)

              Airport From_Airport_Code   Latitude   Longitude
0             ATLANTA               ATL  33.636719  -84.428067
1             BEIJING               PEK  40.080111  116.584556
2              LONDON               LHR  51.477500   -0.461389
3             CHICAGO               ORD  41.978603  -87.904842
4               TOKYO               HND  35.552258  139.779694
5         LOS ANGELES               LAX  33.942536 -118.408075
6               PARIS               CDG  49.012779    2.550000
7   DALLAS/FORT WORTH               DFW  32.896828  -97.037997
8           FRANKFURT               FRA  50.026421    8.543125
9           HONG KONG               HKG  22.308919  113.914603
10             DENVER               DEN  39.861656 -104.673178
11              DUBAI               DXB  25.252778   55.364444
12            JAKARTA               CGK  -6.125567  106.655897
13          AMSTERDAM               AMS  52.308613    4.763889
14             MADRID               MAD  40.493556   -3

Task 1:

In [4]:
###Restrict Airport data set to airport codes and index
airport1=airport.loc[:,'From_Airport_Code']

###Restrict Latlong data set to airport names and corresponding airport codes
latlong1=latlong[['Airport','From_Airport_Code']].set_index('Airport')

###Restrict Airport data set to passenger id and index
passenger1=airport.loc[:,'Passenger_ID']
#
#print(airport1)
#print(latlong1)
#print(passenger1)

In [5]:
#Task 1: Determine the total number of flights from each airport

#Initialise lists
targetlist = []
dictionarylist = []

#common variables
#CountDB: database containing variable and values of interest
#LabelDB: the database containing the new labels you wish to overwrite countdb with.
#MergeVariable: the primary variable which you are replacing with your labeldb
#ThreadNo: Number of Threads
#Replace_Dictionary: Dictionary containing values from LabelDB to replace values in Count DB

#Function:creates a list of indices to partition the CountDB into number of ThreadNo partitions 
def divider(CountDB,ThreadNo): 
  filevalue, ThreadNoValue = divmod(CountDB.size, ThreadNo) 
  #Calculate what the modulo is based on the ThreadNo of the file
  return (CountDB[i*filevalue+min(i, ThreadNoValue):(i+1)*filevalue+min(i+1, ThreadNoValue)] for i in range(ThreadNo))

#Function for partitioning the sets in preperation for threading
def partitioner1(CountDB,ThreadNo,i):
  #tests if CountDB can be cleanly partitioned by split of ThreadNo.
  if CountDB.size % ThreadNo == 0:
      divider(CountDB,ThreadNo)
      listing= list(divider(CountDB,ThreadNo))
      index=listing[i]
      #calls from the list and returns the index of the ith partition
      return index
  else:
    #returns error if CountDB cannot be cleanly partitioned by split of ThreadNo.
    print(str(CountDB.size) + ' cannot be subdivided by '+ str(ThreadNo))
    raise ValueError

#CountDB: file containing index and value which is being counted
def read_input(CountDB):
  for line in CountDB:
  # Split the line into words
    yield line.split()

def inputmerge(LabelDB,MergeVariable):
  #Input Segment
  label_dictionary = LabelDB.to_dict(orient="dict")[MergeVariable] 
  #converts LabelDB into dictionary based on the code you're trying to merge
  inv_label_dictionary = {value: key for key, value in label_dictionary.items()}
  #Inverts dictionary and returns it
  #Output: <Airport, FAA Code> -> <FAA Code, Airport>
  return inv_label_dictionary


#Output: <Passenger code, x> where x is count, then work out max value
def mapper1(LabelDB,CountDB,Replace_Dictionary):
  #Map Segment
  #Output: <FAA Code, x> -> <Airport, x>  where x is count
  
  #read in data
  data = read_input(CountDB)
  for words in data:
    for word in words:
      #check to ensure LabelDB is present, otherwise ignore
      try:
        targetcode= Replace_Dictionary[word] 
      except NameError:#send NameError if there isn't a match
        targetcode= word 
        pass 
      dictkey= {targetcode: 1}#count value of interest and store as dictionary
      dictionarylist.append(dict(dictkey)) #store dictionaries in list

def threadsection1(LabelDB,CountDB,inverse_dictionary,ThreadNo):
  #Begin Timer
  start_time = perf_counter()
  #Create thread list
  threads=[]
  #cycle through number of threads
  for i in range(ThreadNo):
    print("Activate thread "+ str(i+1))
    #Call partition for the tread
    parting=partitioner1(CountDB,ThreadNo,i)
    # start the mapper within the thread
    thread = Thread(target=mapper1(LabelDB,parting,inverse_dictionary))
    thread.start()
    #add thread to thread list
    threads.append(thread)

  #cycle through active threads and instruct them wait until the threads terminate.
  for i in range(len(threads)):
    thread=threads[i]
    thread.join()
    print("End thread "+ str(i+1))
  #Stop timer
  end_time = perf_counter()
  #Calculate difference in start/stop time
  print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')



#Reduce segment
def reducer1():
  #sorter segment, sorts dictionaries alphanumerically pre reduction
  dictionary= sorted(dictionarylist, key= lambda d : list(d.keys()))

  for key, value in dictionary[0].items():  #Starts count on first value and 0
    current_target,current_count = [str(key),0] 

  for diction in dictionary:
    for key, value in diction.items():

      if key ==current_target:
        current_count= current_count+value #If value is same as previous value, increment value+1

      else:
        targetkey= {current_target: current_count} 
        targetlist.append(dict(targetkey)) #store previous count and value in list of dictionary
        current_target = key
        current_count = value # set value to key and set count to 1


def main(LabelDB,CountDB,MergeVariable,ThreadNo):
  #map + reduce segments
  try:
    inverse_dictionary=inputmerge(LabelDB,MergeVariable)
  except NameError: #pass name error if merge isn't possible
    pass
  #map ThreadNo up into threads
  threadsection1(LabelDB,CountDB,inverse_dictionary,ThreadNo)
  
  reducer1()

if __name__ == "__main__":
  main(latlong1,airport1,'From_Airport_Code',4)


print(targetlist)

Activate thread 1
Activate thread 2
Activate thread 3
Activate thread 4
End thread 1
End thread 2
End thread 3
End thread 4
It took  0.01 second(s) to complete.
[{'AMSTERDAM': 15}, {'ATLANTA': 36}, {'BANGKOK': 17}, {'BEIJING': 13}, {'CHARLOTTE': 21}, {'CHICAGO': 33}, {'DALLAS/FORT WORTH': 11}, {'DENVER': 46}, {'GUANGZHOU': 37}, {'HOUSTON': 37}, {'JAKARTA': 27}, {'KUALA LUMPUR': 33}, {'LAS VEGAS': 17}, {'LONDON': 25}, {'MADRID': 13}, {'MIAMI': 11}, {'MUNICH': 14}, {'NEW YORK': 25}, {'PARIS': 21}, {'ROME': 15}, {'SHANGHAI': 20}]


In [6]:
###Task 2: Determine the passenger having had the highest number of flights.

#Initilise lists to store data
targetlist = []
dictionarylist2 = []

#common variables
#CountDB: database containing variable and values of interest

def read_input(CountDB):
  for line in CountDB:
  # Split the line into words
    yield line.split()

#CountDB: file containing index and value which is being counted
#Output: <Passenger code, x> where x is count, then work out max value
def mapper2(CountDB):
  #Map Segment
  #read in data
  inputdata = read_input(CountDB)
  for words in inputdata:
    dictkey2= {word: 1 for word in words} #count value of interest and store as dictionary
    dictionarylist2.append(dict(dictkey2)) #store dictionaries in list
  
#Reduce segment
def reducer2():
  #sorter segment, sorts dictionaries alphanumerically pre reduction
  dictionary2= sorted(dictionarylist2, key= lambda d : list(d.keys()))

  #reduce segment
  for key, value in dictionary2[0].items(): #Starts count on first value and 0
    current_target,current_count = [str(key),0] 

  for diction2 in dictionary2: 
    for key, value in diction2.items(): 

      if key ==current_target: #If value is same as previous value, increment value+1
        current_count= current_count+value

      else: #Else value is not same,
        targetkey= {current_target: current_count} 
        targetlist.append(dict(targetkey)) #store previous count and value in list of dictionary
        current_target = key
        current_count = value # set value to key and set count to 1

def main(CountDB):
  #map + reduce segments
  mapper2(CountDB)
  reducer2()
  mosttarget = max(targetlist, key=lambda d:list(d.values())) #calculate maximum of final list of dictionaries
  print(mosttarget)


if __name__ == "__main__":
  main(passenger1)

#print(targetlist)


{'UES9151GS5': 25}
